In [30]:
!pip install torchmetrics

                                              0.0/729.2 kB ? eta -:--:--
     -                                     30.7/729.2 kB 435.7 kB/s eta 0:00:02
     ----                                  92.2/729.2 kB 871.5 kB/s eta 0:00:01
     ------------                           245.8/729.2 kB 1.7 MB/s eta 0:00:01
     -------------------                    368.6/729.2 kB 1.9 MB/s eta 0:00:01
     ------------------------------         583.7/729.2 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 729.2/729.2 kB 2.7 MB/s eta 0:00:00


In [26]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# graph deep learning
import torch_geometric
import torch_geometric_temporal
import networkx as nx

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
%matplotlib inline

# For Progress Bar
from tqdm.auto import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## get data
+ for x, y in loader shape: (batch_size, in_dim, num_nodes, seq_len)

In [4]:
from Data.generate_data import generate_data, get_data, get_and_normalize_data

train_loader, valid_loader, test_loader, edge_index = generate_data(168, 50, 8)

## train config

In [14]:
from datetime import datetime

train_config = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'n_epochs': 30,  # Number of epochs.
    'learning_rate': 1e-3,
    'early_stop': 400,  # If model has not improved for this many consecutive epochs, stop training.
    'save_path': f'./models/model_{datetime.now()}.ckpt',  # Your model will be saved here.
}

## mtgnn without edge

In [39]:
model_args = {'gcn_true': True, 'buildA_true': True, 'gcn_depth': 2, 'num_nodes': 50, 'dropout': 0.3,
              'subgraph_size': 20, 'node_dim': 40, 'dilation_exponential': 2, 'conv_channels': 16,
              'residual_channels': 16, 'skip_channels': 32, 'end_channels': 64, 'seq_in_len': 168, 'in_dim': 7,
              'seq_out_len': 1,
              'layers': 5, 'propalpha': 0.05, 'tanhalpha': 3, 'layer_norm_affline': True, 'kernel_size': 7,
              'kernel_set': [2, 3, 6, 7]}

In [40]:
from torch_geometric_temporal import MTGNN

net = MTGNN(model_args['gcn_true'], model_args['buildA_true'], model_args['gcn_depth'], model_args['num_nodes'], kernel_set=model_args['kernel_set'],
              kernel_size=model_args['kernel_size'],
              dropout=model_args['dropout'], subgraph_size=model_args['subgraph_size'],
              node_dim=model_args['node_dim'], dilation_exponential=model_args['dilation_exponential'],
              conv_channels=model_args['conv_channels'], residual_channels=model_args['residual_channels'],
              skip_channels=model_args['skip_channels'], end_channels=model_args['end_channels'],
              seq_length=model_args['seq_in_len'], in_dim=model_args['in_dim'], out_dim=model_args['seq_out_len'],
              layers=model_args['layers'], propalpha=model_args['propalpha'], tanhalpha=model_args['tanhalpha'], layer_norm_affline=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = net.to(device)

In [47]:
loss = nn.MSELoss(reduction='mean').to(train_config['device'])

In [42]:
from model.criterions import MapeCriterion, MaeCriterion, RrseCriterion

train_criterion_set = [MapeCriterion(), MaeCriterion(), RrseCriterion()]

valid_criterion_set = [MapeCriterion(), MaeCriterion(), RrseCriterion()]

In [43]:
optim = torch.optim.Adam(model.parameters())

In [44]:
from model.log import Log

log = Log(train_config)

## train model

In [45]:
from model.modle import Model
model = Model(net, loss, train_criterion_set, valid_criterion_set, optim, log, train_config)

In [46]:
model.trainer(train_loader, valid_loader)

  0%|          | 0/315 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "D:\conda\envs\kaggle\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lizhuofeng\AppData\Local\Temp\ipykernel_17868\2202349084.py", line 1, in <module>
    model.trainer(train_loader, valid_loader)
  File "C:\Users\lizhuofeng\Desktop\2023.7.9_service_code\TopoMAD\model\modle.py", line 31, in trainer
    train_loss, pred = self.train_optim(x, y)
  File "C:\Users\lizhuofeng\Desktop\2023.7.9_service_code\TopoMAD\model\modle.py", line 100, in train_optim
    train_loss.backward()
  File "D:\conda\envs\kaggle\lib\site-packages\torch\_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "D:\conda\envs\kaggle\lib\site-packages\torch\autograd\__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, anoth